## 영어 Word2Vec 만들기

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

### 1) 훈련 데이터 이해하기

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", 
                           filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7f527600dad0>)

### 2) 훈련 데이터 전처리하기

In [6]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [8]:
parse_text

Output hidden; open in https://colab.research.google.com to view.

In [9]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [10]:
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


### 3) Word2Vec 훈련
- size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
- window = 컨텍스트 윈도우 크기
- min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 - 학습하지 않는다.)
- workers = 학습을 위한 프로세스 수
- sg = 0은 CBOW, 1은 Skip-gram.

In [11]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

In [26]:
model_result = model.wv.most_similar("man")
model_result

[('woman', 0.8544425368309021),
 ('guy', 0.8135048151016235),
 ('lady', 0.7769224047660828),
 ('girl', 0.7763534784317017),
 ('boy', 0.77060467004776),
 ('gentleman', 0.7324968576431274),
 ('kid', 0.7003461122512817),
 ('poet', 0.6933159828186035),
 ('soldier', 0.6914653778076172),
 ('friend', 0.6530729532241821)]

### 4) Word2Vec 모델 저장하고 로드하기

In [27]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [28]:
model_result = loaded_model.most_similar("man")
model_result

[('woman', 0.8544425368309021),
 ('guy', 0.8135048151016235),
 ('lady', 0.7769224047660828),
 ('girl', 0.7763534784317017),
 ('boy', 0.77060467004776),
 ('gentleman', 0.7324968576431274),
 ('kid', 0.7003461122512817),
 ('poet', 0.6933159828186035),
 ('soldier', 0.6914653778076172),
 ('friend', 0.6530729532241821)]

In [34]:
# man - boy + girl ==> woman
model.wv.most_similar(positive=['man','girl'],negative=['boy'],topn=3)

[('woman', 0.8342835307121277),
 ('guy', 0.7570106983184814),
 ('lady', 0.7363219857215881)]